# 債券プロジェクト

In [1]:
import pandas as pd
import datetime
import numpy as np

債務と債券情報の入力

In [2]:
bond_data = pd.DataFrame(data=[
    [6.625, datetime.date(2022, 2, 15), 100.00],
    [9.125, datetime.date(2022, 2, 15), 100.69],
    [7.875, datetime.date(2022, 8, 15), 100.75],
    [8.250, datetime.date(2022, 8, 15), 101.03],
    [8.250, datetime.date(2023, 2, 15), 101.22],
    [8.375, datetime.date(2023, 2, 15), 101.38],
    [8.000, datetime.date(2023, 8, 15), 100.81],
    [8.750, datetime.date(2023, 8, 15), 102.03],
    [6.875, datetime.date(2024, 2, 15),  98.16],
    [8.875, datetime.date(2024, 2, 15), 102.28],
    [6.875, datetime.date(2024, 8, 15),  97.41],
    [8.625, datetime.date(2024, 8, 15), 101.72],
    [7.750, datetime.date(2025, 2, 15),  99.16],
    [11.25, datetime.date(2025, 2, 15), 109.13],
    [8.500, datetime.date(2025, 8, 15), 101.41],
    [10.50, datetime.date(2025, 8, 15), 107.84],
    [7.875, datetime.date(2026, 2, 15),  99.41],
    [8.875, datetime.date(2026, 2, 15), 103.00],
    ],
    columns=['クーポン', '満期日', '価格'])

debt_data = pd.DataFrame(data=[
    [datetime.date(2022, 2, 15),  2000],
    [datetime.date(2022, 8, 15), 20000],
    [datetime.date(2023, 2, 15),     0],
    [datetime.date(2023, 8, 15), 25000],
    [datetime.date(2024, 2, 15),  1000],
    [datetime.date(2024, 8, 15),     0],
    [datetime.date(2025, 2, 15), 20000],
    [datetime.date(2025, 8, 15),  1000],
    [datetime.date(2026, 2, 15), 15000],
    ],
    columns=['支払日', '債務価格'])

In [3]:
bond_data.head()

,クーポン,満期日,価格
0,6.625,2022-02-15,100.00
1,9.125,2022-02-15,100.69
2,7.875,2022-08-15,100.75
3,8.250,2022-08-15,101.03
4,8.250,2023-02-15,101.22


In [4]:
debt_data.head()

,支払日,債務価格
0,2022-02-15,2000
1,2022-08-15,20000
2,2023-02-15,0
3,2023-08-15,25000
4,2024-02-15,1000


スポットレートカーブを問4-4の結果で定義する。

In [5]:
def r(t):
    alpha = [3.9725e-02, 1.2560e-02, 1.5246e-03, -9.4998e-04, 8.3856e-05]
    return alpha[0] + alpha[1]*t + alpha[2]*t**2 + alpha[3]*t**3 + alpha[4]*t**4

## (a)簡単なキャッシュ・フロー・マッチング
利用可能な債券の数を$m$、$j$番目の債券の価格を$p_j$とする。<br>
債務の生じる期間を$n$期間とし、$i$期間後の債務を$y_i$、$j$番目の債券の$i$期間後のクーポン(とあれば満期返戻価格)を$c_{ij}$とする。<br>
$j$番目の債券の保有量を$x_j$とした時、
\begin{align}
\min_{\bf x}&\sum_{j=1}^mp_jx_j\\
s.t.\ \sum_{j=1}^mc_{ij}x_j\geq& y_i\ \ (i=1,2,\cdots,n)\\
x_j\geq& 0\ \ (j=1,2,\cdots,m)
\end{align}

In [6]:
from pulp import *
m = LpProblem() # 最小化問題として定式化
x = [LpVariable('x{0}'.format(i+1), lowBound=0) for i in range(len(bond_data))] # 正の変数xの設定
m += (bond_data['価格']*x).sum() # 目的関数
for i, t in enumerate(debt_data['支払日']):
    # i期間後に生じる債務
    y = debt_data['債務価格'][i]
    # それぞれの債券でi期間後に得られるリターン
    c = bond_data['クーポン'].where(bond_data['満期日']>=t, 0) + (bond_data['満期日']==t)*100
    # 制約条件
    m += (c*x).sum() >= y

In [7]:
m.solve() # ソルバーの実行

1

In [8]:
for i in range(len(x)):
    print(x[i], value(x[i]))

x1 0.0
x2 0.0
x3 0.0
x4 145.32327
x5 0.0
x6 0.0
x7 0.0
x8 207.31244
x9 0.0
x10 0.0
x11 0.0
x12 0.0
x13 175.45227
x14 0.0
x15 0.0
x16 0.0
x17 139.04983
x18 0.0


これが費用最小のポートフォリオとなる。総費用は、

In [9]:
sum([value(x[i])*bond_data['価格'][i] for i in range(len(bond_data))])

67054.8889148

## (b)複雑なキャッシュ・フロー・マッチング
まずはそれぞれの債券のキャッシュ・フローを書き下す（教科書の表5.3のイメージ）

In [10]:
data = [[datetime.date(2021,8,15)] + list(-bond_data['価格'])]
for i, t in enumerate(debt_data['支払日']):
    datum = [t]
    for j in bond_data.itertuples():
        datum.append(j.クーポン if j.満期日>t else j.クーポン+100 if j.満期日==t else 0)
    data.append(datum)

cash_flow = pd.DataFrame(data, columns=['年']+['実在債券{0}'.format(i+1) for i in range(len(bond_data))])#.set_index('年')
cash_flow

,年,実在債券1,実在債券2,実在債券3,実在債券4,実在債券5,実在債券6,実在債券7,実在債券8,実在債券9,実在債券10,実在債券11,実在債券12,実在債券13,実在債券14,実在債券15,実在債券16,実在債券17,実在債券18
0,2021-08-15,-100.000,-100.690,-100.750,-101.03,-101.22,-101.380,-100.81,-102.03,-98.160,-102.280,-97.410,-101.720,-99.16,-109.13,-101.41,-107.84,-99.410,-103.000
1,2022-02-15,106.625,109.125,7.875,8.25,8.25,8.375,8.00,8.75,6.875,8.875,6.875,8.625,7.75,11.25,8.50,10.50,7.875,8.875
2,2022-08-15,0.000,0.000,107.875,108.25,8.25,8.375,8.00,8.75,6.875,8.875,6.875,8.625,7.75,11.25,8.50,10.50,7.875,8.875
3,2023-02-15,0.000,0.000,0.000,0.00,108.25,108.375,8.00,8.75,6.875,8.875,6.875,8.625,7.75,11.25,8.50,10.50,7.875,8.875
4,2023-08-15,0.000,0.000,0.000,0.00,0.00,0.000,108.00,108.75,6.875,8.875,6.875,8.625,7.75,11.25,8.50,10.50,7.875,8.875
5,2024-02-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,106.875,108.875,6.875,8.625,7.75,11.25,8.50,10.50,7.875,8.875
6,2024-08-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,0.000,106.875,108.625,7.75,11.25,8.50,10.50,7.875,8.875
7,2025-02-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,0.000,0.000,0.000,107.75,111.25,8.50,10.50,7.875,8.875
8,2025-08-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.00,0.00,108.50,110.50,7.875,8.875
9,2026-02-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.00,107.875,108.875


ここに、余剰資金を運用するための人工的な債券のキャッシュ・フローを追加する。<br>
連続金利でのフォワードレートは
\begin{align}
f_{t_1,t_2} = \frac{r(t_2)t_2-r(t_1)t_1}{t_2-t_1}
\end{align}
である。ただし、現在(2021/8/15)から$i$期間後までの時間を$t_i$年とする。<br>
なので、例えば$i$期間後から$i+1$期間後までの間の再投資を表現するキャッシュ・フローは
\begin{align}
(0,0,\cdots,0,-1,1+f_{t_i,t_{i+1}},0,\cdots,0 )
\end{align}

In [11]:
t = cash_flow['年']
for i in range(len(cash_flow)-1):
    ti = (t[i]-t[0]).days/365
    ti_ = (t[i+1]-t[0]).days/365
    datum = [0]*(len(debt_data)+1)
    datum[i] = -1
    datum[i+1] = 1 + (r(ti_)*ti_ - r(ti)*ti)/(ti_ - ti)
    cash_flow['人工債券{0}'.format(i+1)]=datum
cash_flow

,年,実在債券1,実在債券2,実在債券3,実在債券4,実在債券5,実在債券6,実在債券7,実在債券8,実在債券9,...,実在債券18,人工債券1,人工債券2,人工債券3,人工債券4,人工債券5,人工債券6,人工債券7,人工債券8,人工債券9
0,2021-08-15,-100.000,-100.690,-100.750,-101.03,-101.22,-101.380,-100.81,-102.03,-98.160,...,-103.000,-1.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
1,2022-02-15,106.625,109.125,7.875,8.25,8.25,8.375,8.00,8.75,6.875,...,8.875,1.046328,-1.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,2022-08-15,0.000,0.000,107.875,108.25,8.25,8.375,8.00,8.75,6.875,...,8.875,0.000000,1.059669,-1.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,2023-02-15,0.000,0.000,0.000,0.00,108.25,108.375,8.00,8.75,6.875,...,8.875,0.000000,0.000000,1.071797,-1.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,2023-08-15,0.000,0.000,0.000,0.00,0.00,0.000,108.00,108.75,6.875,...,8.875,0.000000,0.000000,0.000000,1.081134,-1.0000,0.000000,0.000000,0.000000,0.000000
5,2024-02-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,106.875,...,8.875,0.000000,0.000000,0.000000,0.000000,1.0867,-1.000000,0.000000,0.000000,0.000000
6,2024-08-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,...,8.875,0.000000,0.000000,0.000000,0.000000,0.0000,1.088183,-1.000000,0.000000,0.000000
7,2025-02-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,...,8.875,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,1.085847,-1.000000,0.000000
8,2025-08-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,...,8.875,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,1.080656,-1.000000
9,2026-02-15,0.000,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.000,...,108.875,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1.074159


この状態で(a)と同じ最適化を行う。以下再掲

利用可能な債券の数を$m$、$j$番目の債券の価格を$p_j$とする。<br>
債務の生じる期間を$n$期間とし、$i$期間後の債務を$y_i$、$j$番目の債券の$i$期間後のクーポン(とあれば満期返戻価格)を$c_{ij}$とする。<br>
$j$番目の債券の保有量を$x_j$とした時、
\begin{align}
\min_{\bf x}&\sum_{j=1}^mp_jx_j\\
s.t.\ \sum_{j=1}^mc_{ij}x_j\geq& y_i\ \ (i=1,2,\cdots,n)\\
x_j\geq& 0\ \ (j=1,2,\cdots,m)
\end{align}

In [12]:
m = LpProblem() # 最小化問題として定式化
x = [LpVariable('x{0}'.format(i+1), lowBound=0) for i in range(len(cash_flow.columns)-1)] # 正の変数xの設定
m += (np.array(-cash_flow[cash_flow.index==0].loc[:, cash_flow.columns.str.contains('債券')])*x).sum()  # 目的関数
for i, t in enumerate(debt_data['支払日']):
    # i期間後に生じる債務
    y = debt_data['債務価格'][i]
    # それぞれの債券でi期間後に得られるリターン
    c = np.array(cash_flow[cash_flow['年']==t].loc[:, cash_flow.columns.str.contains('債券')])
    # 制約条件
    m += (c*x).sum() >= y

In [13]:
m.solve() # ソルバーの実行


1

In [14]:
for i in range(len(x)):
    print(x[i], value(x[i]))

x1 0.0
x2 0.0
x3 0.0
x4 113.57254
x5 0.0
x6 0.0
x7 0.0
x8 160.75726
x9 0.0
x10 0.0
x11 0.0
x12 0.0
x13 0.0
x14 256.13805
x15 0.0
x16 0.0
x17 0.0
x18 0.0
x19 0.0
x20 3225.1526
x21 0.0
x22 4288.1791
x23 0.0
x24 1881.5531
x25 4929.0278
x26 13847.526
x27 13964.411


実際の債券のポートフォリオのみ表示すれば、

In [15]:
for i in range(len(bond_data)):
    print(x[i], value(x[i]))

x1 0.0
x2 0.0
x3 0.0
x4 113.57254
x5 0.0
x6 0.0
x7 0.0
x8 160.75726
x9 0.0
x10 0.0
x11 0.0
x12 0.0
x13 0.0
x14 256.13805
x15 0.0
x16 0.0
x17 0.0
x18 0.0


これが費用最小のポートフォリオとなる。総費用は、

In [16]:
sum([value(x[i])*bond_data['価格'][i] for i in range(len(bond_data))])

55828.6423505

## (c)デュレーション・マッチング
期間$i$における$j$番目の債券のキャッシュフローを$c_{ij}$、債務のキャッシュフローを$y_i$とする。<br>
また現在から$i$期間後までの時間を$t_i$年とする。

In [17]:
# 時間
t = np.array([(cash_flow['年'][i] - cash_flow['年'][0]).days/365 for i in range(len(cash_flow))])
# 債券のキャッシュフローを格納した行列
c = np.array(cash_flow.loc[:, cash_flow.columns.str.contains('実在')])
# 債務のキャッシュフローを格納した行列
y = np.append(0, np.array(debt_data['債務価格']))

$j$番目の債券の現在価値を$PV_j$、債務の現在価値を$PV$とすると、
\begin{align}
PV_j = \sum_{i=1}^nc_{ij}e^{-r(t_i)t_i}\\
PV = \sum_{i=1}^ny_ie^{-r(t_i)t_i}
\end{align}

In [18]:
# 債券の現在価値(bond)
PVb = (np.exp(-r(t[1:])*t[1:])*c[1:].T).sum(axis=1)
# 債務の現在価値(debt)
PVd = (np.exp(-r(t[1:])*t[1:])*y[1:]).sum()

また、ケース$k$として、期間構造のパラメータ$\alpha_k$の変化に対して守る場合、$j$番目の債券のデュレーションを$D^{(k)}_j$、債務のデュレーションを$D^{(k)}$とすると、
\begin{align}
D^{(k)}_j &= -\frac{1}{PV_j}\frac{d}{d\alpha_k}PV_j\\
&=-\frac{1}{PV_j}\sum_{i=1}^nc_{ij}\frac{d}{d\alpha_k}e^{-r(t_i)t_i}\\
&=-\frac{1}{PV_j}\sum_{i=1}^nc_{ij}e^{-r(t_i)t_i}(-t_i^kt_i)\\
&= \frac{1}{PV_j}\sum_{i=1}^nt_i^{k+1}c_{ij}e^{-r(t_i)t_i}\\
D^{(k)} &= -\frac{1}{PV}\frac{d}{d\alpha_k}PV\\
 &= -\frac{1}{PV}\frac{d}{d\alpha_k}\sum_{i=1}^ny_ie^{-r(t_i)t_i}\\
 &= \frac{1}{PV}\sum_{i=1}^nt_i^{k+1}y_ie^{-r(t_i)t_i}\\
\end{align}

In [19]:
# 債券のデュレーション
Db = np.empty((5, len(bond_data)))
for k in range(5):
    Db[k] = (t**(k+1)*c.T*np.exp(-r(t)*t)).sum(axis=1)/PVb
# 債務のデュレーション
Dd = np.empty((5))
for k in range(5):
    Dd[k] = (t**(k+1)*y*np.exp(-r(t)*t)).sum()/PVd

したがって$j$番目の債券の購入単位を$x_j$とすると、ケース$k$の場合におけるイミュナイズの方程式は、
\begin{align}
\sum_{j=1}^mx_jPV_j &= PV\\
\sum_{j=1}^mx_jPV_jD^{(k)}_j &= PVD^{(k)}\\
\end{align}

これを制約として、初期の購入価格を最小にする最小問題を解けば良い。
\begin{align}
\min_{\bf x}\sum_{j=1}^mx_jp_j
\end{align}

In [20]:
ans_df = pd.DataFrame(data=[], columns=['ケース{0}'.format(k+1) for k in range(5)], index=['x{0}'.format(j+1) for j in range(len(bond_data))]+['費用'])
for k in range(5):
    m = LpProblem() # 最小化問題として定式化
    x = [LpVariable('x{0}'.format(i+1), lowBound=0) for i in range(len(bond_data))] # 正の変数xの設定
    m += (bond_data['価格']*x).sum() # 目的関数
    m += (PVb*x).sum() == PVd # 現在価値の制約
    m += (Db[k]*PVb*x).sum() == PVd*Dd[k] # デュレーションの制約
    m.solve() # ソルバーの実行

    ans_df['ケース{0}'.format(k+1)] = [value(x[i]) for i in range(len(x))]+[([value(x[i]) for i in range(len(x))]*bond_data['価格']).sum()]

In [21]:
ans_df

,ケース1,ケース2,ケース3,ケース4,ケース5
x1,0.000000,0.000000,0.000000,0.000000,0.000000
x2,97.706900,0.000000,0.000000,0.000000,0.000000
x3,0.000000,0.000000,0.000000,0.000000,0.000000
x4,0.000000,0.000000,0.000000,0.000000,0.000000
x5,0.000000,0.000000,0.000000,0.000000,0.000000
x6,0.000000,0.000000,0.000000,0.000000,0.000000
x7,0.000000,0.000000,0.000000,0.000000,0.000000
x8,0.000000,130.309230,0.000000,0.000000,0.000000
x9,0.000000,0.000000,0.000000,0.000000,0.000000
x10,0.000000,0.000000,66.926931,0.000000,0.000000


これがそれぞれのパターンにおいてイミュナイズされたポートフォリオの構成である。